In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa as lr
import librosa.display
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
#import lightgbm as lgb
#from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
def audio_plot(audio_file):
    audio, Fs = lr.load(audio_file) #Fs is the sampling rate

    #print(audio.size/Fs) #gives the time duration of the audio signal
    #print(Fs)

    Ts = 1.0/Fs #Time step of a single sample
    t = numpy.arange(0, len(audio)/Fs, Ts) #time vector

    N = len(audio) #total number of samples
    #k = numpy.arange(N)
    #T = N/Fs #time duration of the audio signal
    #f = k/T #since I need total 'k' samples in the time duration of the audio
    f = numpy.fft.fftfreq(N, d=Ts)

    fft_audio = numpy.fft.fft(audio)/N

    fig, ax = plt.subplots(2, 1, figsize=(12, 4))
    ax[0].plot(t, audio)
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Amplitude')
    
    ax[1].plot(f/1000, abs(fft_audio)) 
    ax[1].set_xlabel('Freq (KHz)')
    ax[1].set_ylabel('|Y(freq)|')

In [ ]:
n_class = 9
n_aud_files = 67

files = numpy.empty([n_class, n_aud_files], dtype=object)

data_folder = glob.glob("../Read_Up/*")
#print(data_folder)

for (i, session) in zip(range(n_class), data_folder):
    for (j, aud_file) in zip(range(n_aud_files), glob.glob("%s/*"%session)):
        files[i][j] = aud_file
                           

In [ ]:
audio_plot('../Read_Up/IDR1/1.wav')

In [7]:
audio, Fs = lr.load('../Read_Up/IDR1/06.wav') #fs = 22050
mfcc_feat = mfcc(audio, Fs, winlen=0.023)
d_mfcc_feat = delta(mfcc_feat, 2)
fbank_feat = logfbank(audio, Fs, winlen=0.023)

#551=Fs*25ms

print(mfcc_feat.shape)
print(fbank_feat.shape)
print(d_mfcc_feat.shape)

print(np.var(mfcc_feat, axis=0))

(866, 13)
(866, 26)
(866, 13)
[  9.0981951  263.80518191 194.81658139 328.84020909 200.83098183
 471.91991385 155.96378967 410.56157023 187.31141606 272.79651232
 133.62223024 136.64788234  83.83187247]


In [35]:
audio, Fs = lr.load('../Read_Up/IDR1/06.wav') #fs = 22050
chroma = lr.feature.chroma_stft(y=audio, sr=Fs, n_fft=512)
mfcc = lr.feature.mfcc(y=audio, sr=Fs, n_mfcc=13, n_fft=512)
melspectro = lr.feature.melspectrogram(y=audio, sr=Fs, n_fft=512, n_mels=40)
rmse_coeff = lr.feature.rmse(y=audio, frame_length=512)

print(np.mean(rmse_coeff.T, axis=0))

[0.00797312]


In [ ]:
audio_files = []
for i in range(n_class):
    for j in range(n_aud_files):
        aud, Fs = lr.load(files[i][j])
        audio_files.append([aud, Fs, i])
        


In [ ]:
def get_features(audio_files):
    output = []
    for i in audio_files:
        aud = i[0]
        rate = i[1]
        label = i[2]
        mfcc_array = mfcc(aud, rate, winlen=0.023)
        n = mfcc_array.shape[0]
        m = mfcc_array.shape[1]
        final_mfcc = []
        for j in range(m):
            tot = 0
            for i in range(n):
                tot += mfcc_array[i][j]
            final_mfcc.append((tot*1.0)/n)
        final_mfcc.append(label)
        output.append(final_mfcc)
    return output


inp = get_features(audio_files)
train, test = train_test_split(inp, test_size=0.3, random_state=0)

In [ ]:
train = numpy.array(train)
test = numpy.array(test)

In [ ]:
x_train = numpy.delete(train,13,1)
y_train = train[:, 13]

x_test = numpy.delete(test,13,1)
y_test = test[:, 13]

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
model = LogisticRegression(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

In [ ]:
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_test,y_test)

In [ ]:
model = svm.SVC()
model.fit(x_train, y_train)
model.score(x_test, y_test)

In [ ]:
model = tree.DecisionTreeClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

In [ ]:
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(x_train, y_train)
model.score(x_test,y_test)

In [ ]:
model= RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

In [ ]:
model = AdaBoostClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

In [ ]:
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

In [ ]:
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_test,y_test)